In [35]:
import threading, requests, time
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import pandas as pd
from konlpy.tag import Twitter
from collections import Counter 
from pydub import AudioSegment
from pydub.playback import play
import argparse
from moviepy.editor import *
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import time
import pyaudio
import wave
from array import array

twitter = Twitter()

xx = pd.read_csv("욕욕.csv",names = ["욕"],encoding='cp949')
vect = CountVectorizer(tokenizer=twitter.nouns)
vect.fit(xx['욕'])

GOOGLE_APPLICATION_CREDENTIALS = "voice-be4b7165be4c.json"

def save_wav(day,count):
    now = day 
    FORMAT=pyaudio.paInt16
    CHANNELS=1
    RATE=44100
    CHUNK=int(RATE / 10)
    RECORD_SECONDS=8
    FILE_NAME="RECORDING_{}_{}_{}_{}.wav".format(now.tm_year, now.tm_mon, now.tm_mday,count)

    audio=pyaudio.PyAudio()

    stream=audio.open(format=FORMAT,channels=CHANNELS,
                          rate=RATE,
                          input=True,
                          frames_per_buffer=CHUNK)

    frames=[]

    for i in range(0,int(RATE/CHUNK*RECORD_SECONDS)):
        data=stream.read(CHUNK)
        data_chunk=array('h',data)
        vol=max(data_chunk)
        if(vol>=500):
            print("something said")
            frames.append(data)
        else:
            print("nothing")
        print("\n")


    stream.stop_stream()
    stream.close()
    audio.terminate()
    wavfile=wave.open(FILE_NAME,'wb')
    wavfile.setnchannels(CHANNELS)
    wavfile.setsampwidth(audio.get_sample_size(FORMAT))
    wavfile.setframerate(RATE)
    wavfile.writeframes(b''.join(frames))

def wav_processing(day,count):
    now = day
    CHANNELS=1
    client = speech.SpeechClient()
    wav_file = "RECORDING_{}_{}_{}_{}.wav".format(now.tm_year, now.tm_mon, now.tm_mday,count)

    with io.open(wav_file, 'rb') as audio_file:
        content = audio_file.read()            

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,audio_channel_count=CHANNELS,
        language_code='ko-KR',
        enable_word_time_offsets=True)

    operation = client.long_running_recognize(config, audio)


    print('Waiting for operation to complete...')
    result = operation.result(timeout=90)

    word_list = []
    start_time_list = []
    end_time_list = []

    for k in result.results:
        alternatives = k.alternatives
        for alternative in alternatives:
            print('Transcript: {}'.format(alternative.transcript))
            print('Confidence: {}'.format(alternative.confidence))

        for word_info in alternative.words:
            wo = word_info.word
            word_list.append(wo)
            start_time = word_info.start_time
            end_time = word_info.end_time
            start_time_list.append(start_time.seconds + start_time.nanos * 1e-9)
            end_time_list.append(end_time.seconds + end_time.nanos * 1e-9)
            print('Word: {}, start_time: {}, end_time: {}'.format(
                wo,
                start_time.seconds + start_time.nanos * 1e-9,
                end_time.seconds + end_time.nanos * 1e-9))

    sentences_tag = []
    for sentence in word_list:
        morph = twitter.pos(sentence)
        sentences_tag.append(morph)

    noun_adj_list = []
    for sentence1 in sentences_tag:
        for word, tag in sentence1:
            if tag in ['Noun','Adjective']:
                noun_adj_list.append(word)

    word_vect = vect.transform(noun_adj_list)
    feature_list = []

    for i in word_vect.toarray():
        for j in range(0,len(i)):
            if i[j] == 1:
                feature_list.append(vect.get_feature_names()[j])


    sound1 = AudioSegment.from_file(wav_file)


    for i in range(0,len(word_list)):
        for j in range(0,len(feature_list)):
            if feature_list[j] in word_list[i]:
                print(start_time_list[i])
                print(end_time_list[i])
                sound2 = AudioSegment.from_file("Beep2.wav")
                if end_time_list[i]-start_time_list[i]>2.5:
                    sound2 = sound2[start_time_list[i]*1000:(end_time_list[i]*1000)]
                    sound1 = sound1.overlay(sound2,position=(1000*start_time_list[i])+2500,gain_during_overlay =-100)
                else :
                    sound2 = sound2[start_time_list[i]*1000:(end_time_list[i]*1000)]
                    sound1 = sound1.overlay(sound2,position=1000*start_time_list[i],gain_during_overlay =-100)



    sound1.export("yok_{}_{}_{}_{}.wav".format(now.tm_year, now.tm_mon, now.tm_mday,count), format='wav')
 


if __name__ == "__main__":
    day = time.gmtime(time.time())
    count = 0
    for i in range(1):
#     while True:
        if count == 0 :
            t1 = threading.Thread(target=save_wav, args=(day,count))
            t2 = threading.Thread(target = wav_processing, args=(day,count))
            t1.daemon = True 
            t2.daemon = True
            t1.start()

            t1.join()
            t2.start()
            t2.join()
            count += 1
            time.sleep(5)
            rec = AudioSegment.from_file("yok_2019_8_20_0.wav")
            play(rec)
        else:
            t1 = threading.Thread(target=save_wav, args=(day,count))
            t2 = threading.Thread(target = wav_processing, args=(day,count))
            t1.daemon = True 
            t2.daemon = True
            t1.start()
            t1.join()
            t2.start()
            count += 1

nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


nothing


something said


something said


something said


nothing


something said


nothing


something said


something said


something said


something said


something said


something said


something said


something said


something said


something said


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


nothing


not

In [36]:
rec = AudioSegment.from_file("yok_2019_8_20_0.wav")
play(rec)